In [7]:
import pandas as pd
from RNA import RNAconsensus
from pathlib import Path
from src.extract_ss_text_file import extract_ss_text_file
from src.extract_tRNAscan_file import extract_tRNAscan_file
from src.trna_struct import (
    check_struc_abs,
    count_stems,
    find_loop_dots,
    struct_abstractor,
)

# Section 2 | Recovering tRNA candidates (from non-canonical dataset)

## Read in data

In [8]:
# Read in non-canonical tRNAs
df_non_canonical = pd.read_csv("./data_sets/non_canonical_tRNAs.csv")
df_non_canonical

,superphyla,id,name,type,anticodon,seq,length,possible_intron_n,possible_intron_rel,possible_intron_abs,dot_bracket_sec_str,tRNA_structure_abstracted,stem_count,canonical,tRNA_structure_abstracted_viennarna,canonical_viennarna
0,Asgard,3045163_7,JASEJC010000005.trna5,Pro,TGG,GGGGCATTGGTGTAATGGTAGCACCCCTGGTCTGGCACCtAGGAGG...,72,0,[],[],(((((.(..((((.......)))).(((((.......)).)))......,(.(.(.).(..).(.)).).,5,tRNA_out,[[][][]],tRNA_4-stems
1,Euryarchaeota,2026739_441,JAENZL010000013.trna4,Cys,GCA,GCCAAGATGGCGGAGCGGCTACGCAATCGCCTGCAGAGCGATACCa...,72,0,[],[],((((((...(((.........))).(((((.......))))).......,(.(.).(.).(.).).,4,tRNA_out,[[][][]],tRNA_4-stems
2,Euryarchaeota,2026739_441,JAENZL010000028.trna1,Gln,CTG,AGCCCGGTAGTGTAGTGGTcaATCATGCGGGACTCTGACTCCCGCA...,73,0,[],[],(((((((..(((...........))).(((((.......))))).....,(.(.).(.).(.(.).)).,5,tRNA_out,[[][][]],tRNA_4-stems
3,Asgard,2053489_161,JAGXOA010000009.trna3,Ser,TGA,GCAGGTGTAGCCTAGCTGGTAAGACGCTGGCCTTGAGAGCCTGTGC...,83,1,"[[38, 55]]","[[211, 194]]",(((((((..(.((........)).).(.(((.......))).)..(...,(.(.(.).).(.(.).).(.).(.)).,7,tRNA_out,[[][][][]],tRNA_5-stems
4,Asgard,2053489_161,JAGXOA010000027.trna3,Leu,CAA,GCGGGAATGGCTGAGTGGTaAAAGCAATAGATTCAAAATCTATCCT...,83,0,[],[],((((((...(((..........))).(((((.......)))))(((...,(.(.).(.)(.).(.).).,5,tRNA_out,[[][][][]],tRNA_5-stems
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Asgard,2053491_86,JAGXKG010000001.trna14,Gly,CCC,GCGGCCGTGGTCTAGCATGGTTAGGACTGAAGCTTCCCAAGCTTCC...,77,1,"[[55, 78]]","[[285208, 285231]]",(((((((..((((..........)))).(((((.......)))))....,(.(.).(.).(.(.).)).,5,tRNA_out,[[][][]],tRNA_4-stems
130,Asgard,2053491_86,JAGXKG010000001.trna18,Thr,GGT,GCCCCAGTAGCTCAGCCtGGTtAGATCGCATCCTTGGTAAGGATGA...,78,1,"[[40, 69]]","[[399968, 399997]]",(((((((..(.((..........)).).(((((.......)))))....,(.(.(.).).(.).(.)).,5,tRNA_out,[[][][]],tRNA_4-stems
131,Asgard,2053491_86,JAGXKG010000001.trna21,Leu,CAA,GCCGAGGTAGCCAAGCCcGGCcaACGGCGACGGACTCAAGATCCGT...,88,1,"[[41, 66]]","[[430301, 430276]]",(((((((..(((.............))).(((((.......)))))...,(.(.).(.).(.).(.(.).)).,6,tRNA_out,[[][][][]],tRNA_5-stems
132,Asgard,2053491_86,JAGXKG010000001.trna26,Glu,CTC,GAATGTAGTCcGGCccAGCATGGGGGACTCTCGATCCTCCGACCCG...,71,0,[],[],(.((((...........)))).(((((.......)))))....(((...,(.(.).(.).(.).).,4,tRNA_out,[[][][]],tRNA_4-stems


In [9]:
# Readin in anticodon and consensus structures
anticodon_consensus_df = pd.read_csv("./alifold_consensus_results/anticodon_consensus_structures.csv")
anticodon_consensus_df

,anticodon,consensus_structure
0,CTC,(((((((..(((...............)))((((((.......)))...
1,GAC,(((((((..(((((((...)))).))).(((((.......)))))....
2,GTT,(((((((..((((.........)))).(((((.......))))).....
3,TAA,(((((((..(((.............)))(((((((....).)))))...
4,CAA,(((((((..(((.............)))(((((((....).)))))...
5,CAT,(((((((..((((............)))).(((((.......))))...
6,CCG,(((((((..((((............)))).((((((....).))))...
7,GTG,(((((((..((((...........))))(((((((....).)))))...
8,TGG,(((((((..(((((((...).))).)))((((((((...)))))))...
9,CCC,(((((((..((((...........))))((((((.......)))))...


In [10]:
# Create a mapping from anticodon to consensus structure
consensus_map = dict(zip(anticodon_consensus_df["anticodon"], anticodon_consensus_df["consensus_structure"]))

# Add a new column with the consensus structure (or None if not found)
df_non_canonical.loc[:, "consensus_structure"] = df_non_canonical["anticodon"].map(consensus_map)


## Align with consensus structure

In [11]:
# Drop NNN tRNAs and missing consensus structures
df_non_canonical_drop = df_non_canonical.dropna(subset=["consensus_structure"],axis=0)

In [13]:
# Per anticodon
anticodon_unique = df_non_canonical_drop["anticodon"].unique()

for anticodon in anticodon_unique:
    filtered = df_non_canonical_drop[df_non_canonical_drop["anticodon"] == anticodon]
    
    with open(f"./non_canon_refolding_consensus_results/{anticodon}_non_canon_consensus.fasta", mode="w") as fasta:
        fasta.writelines(
            f">{superphyla}__{id}__{aa}_{anticodon}__{name}__{length}bp\n{seq}\n{struct} #FS\n"
            for superphyla, id, aa, anticodon, name, length, seq, struct in zip(filtered["superphyla"], 
                                                                filtered["id"], 
                                                                filtered["type"],
                                                                filtered["anticodon"],
                                                                filtered["name"],
                                                                filtered["length"],
                                                                # filtered["isotype_score"],
                                                                filtered["seq"],
                                                                filtered["consensus_structure"]
                                                                )
        )

Run mlocarna

##  Apply constraints

In [14]:
df_non_canonical

,superphyla,id,name,type,anticodon,seq,length,possible_intron_n,possible_intron_rel,possible_intron_abs,dot_bracket_sec_str,tRNA_structure_abstracted,stem_count,canonical,tRNA_structure_abstracted_viennarna,canonical_viennarna,consensus_structure
0,Asgard,3045163_7,JASEJC010000005.trna5,Pro,TGG,GGGGCATTGGTGTAATGGTAGCACCCCTGGTCTGGCACCtAGGAGG...,72,0,[],[],(((((.(..((((.......)))).(((((.......)).)))......,(.(.(.).(..).(.)).).,5,tRNA_out,[[][][]],tRNA_4-stems,(((((((..(((((((...).))).)))((((((((...)))))))...
1,Euryarchaeota,2026739_441,JAENZL010000013.trna4,Cys,GCA,GCCAAGATGGCGGAGCGGCTACGCAATCGCCTGCAGAGCGATACCa...,72,0,[],[],((((((...(((.........))).(((((.......))))).......,(.(.).(.).(.).).,4,tRNA_out,[[][][]],tRNA_4-stems,(((((((..(((.............)))(((((((....).)))))...
2,Euryarchaeota,2026739_441,JAENZL010000028.trna1,Gln,CTG,AGCCCGGTAGTGTAGTGGTcaATCATGCGGGACTCTGACTCCCGCA...,73,0,[],[],(((((((..(((...........))).(((((.......))))).....,(.(.).(.).(.(.).)).,5,tRNA_out,[[][][]],tRNA_4-stems,(((((((..(((...........)))((((((.......))))))....
3,Asgard,2053489_161,JAGXOA010000009.trna3,Ser,TGA,GCAGGTGTAGCCTAGCTGGTAAGACGCTGGCCTTGAGAGCCTGTGC...,83,1,"[[38, 55]]","[[211, 194]]",(((((((..(.((........)).).(.(((.......))).)..(...,(.(.(.).).(.(.).).(.).(.)).,7,tRNA_out,[[][][][]],tRNA_5-stems,(((((((..(((.............)))((((((.......)))))...
4,Asgard,2053489_161,JAGXOA010000027.trna3,Leu,CAA,GCGGGAATGGCTGAGTGGTaAAAGCAATAGATTCAAAATCTATCCT...,83,0,[],[],((((((...(((..........))).(((((.......)))))(((...,(.(.).(.)(.).(.).).,5,tRNA_out,[[][][][]],tRNA_5-stems,(((((((..(((.............)))(((((((....).)))))...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Asgard,2053491_86,JAGXKG010000001.trna14,Gly,CCC,GCGGCCGTGGTCTAGCATGGTTAGGACTGAAGCTTCCCAAGCTTCC...,77,1,"[[55, 78]]","[[285208, 285231]]",(((((((..((((..........)))).(((((.......)))))....,(.(.).(.).(.(.).)).,5,tRNA_out,[[][][]],tRNA_4-stems,(((((((..((((...........))))((((((.......)))))...
130,Asgard,2053491_86,JAGXKG010000001.trna18,Thr,GGT,GCCCCAGTAGCTCAGCCtGGTtAGATCGCATCCTTGGTAAGGATGA...,78,1,"[[40, 69]]","[[399968, 399997]]",(((((((..(.((..........)).).(((((.......)))))....,(.(.(.).).(.).(.)).,5,tRNA_out,[[][][]],tRNA_4-stems,(((((((..((((..........)))).(((((((...)))))))....
131,Asgard,2053491_86,JAGXKG010000001.trna21,Leu,CAA,GCCGAGGTAGCCAAGCCcGGCcaACGGCGACGGACTCAAGATCCGT...,88,1,"[[41, 66]]","[[430301, 430276]]",(((((((..(((.............))).(((((.......)))))...,(.(.).(.).(.).(.(.).)).,6,tRNA_out,[[][][][]],tRNA_5-stems,(((((((..(((.............)))(((((((....).)))))...
132,Asgard,2053491_86,JAGXKG010000001.trna26,Glu,CTC,GAATGTAGTCcGGCccAGCATGGGGGACTCTCGATCCTCCGACCCG...,71,0,[],[],(.((((...........)))).(((((.......)))))....(((...,(.(.).(.).(.).).,4,tRNA_out,[[][][]],tRNA_4-stems,(((((((..(((...............)))((((((.......)))...


### Make directory per constraint

In [15]:
constrain = "secondUconstr"

In [16]:
output_dir = Path(f"./non_canon_refolding_{constrain}")
output_dir.mkdir(parents=True, exist_ok=True)

In [17]:
# Adapt if needed
with open(f"{output_dir}/non_canonical_for_refolding_{constrain}.fasta", "w") as f:
    counter = 0
    for _, row in df_non_canonical.iterrows():
        header = f"{row['superphyla']}__{row['id']}__{row['name']}"
        seq = row['seq']
        
        # Writing constraints
        # Replace every character with a dot
        constrained = ['.'] * len(seq)
        # constrained[8] = 'x' if seq[8] in ["A", "G"] else "."

        # After position 13, replace first two 'T' with 'x'
        t_count = 0
        for i in range(13, len(seq)):
            if seq[i] == 'T':
                t_count += 1
            
                if t_count < 3 and t_count > 1:
                    constrained[i] = 'x'
        

        constrained = ''.join(constrained)
        
        f.write(f">{header}\n{seq}\n{constrained}\n")
        print(f"{counter} tRNA was processed")
        counter += 1

0 tRNA was processed
1 tRNA was processed
2 tRNA was processed
3 tRNA was processed
4 tRNA was processed
5 tRNA was processed
6 tRNA was processed
7 tRNA was processed
8 tRNA was processed
9 tRNA was processed
10 tRNA was processed
11 tRNA was processed
12 tRNA was processed
13 tRNA was processed
14 tRNA was processed
15 tRNA was processed
16 tRNA was processed
17 tRNA was processed
18 tRNA was processed
19 tRNA was processed
20 tRNA was processed
21 tRNA was processed
22 tRNA was processed
23 tRNA was processed
24 tRNA was processed
25 tRNA was processed
26 tRNA was processed
27 tRNA was processed
28 tRNA was processed
29 tRNA was processed
30 tRNA was processed
31 tRNA was processed
32 tRNA was processed
33 tRNA was processed
34 tRNA was processed
35 tRNA was processed
36 tRNA was processed
37 tRNA was processed
38 tRNA was processed
39 tRNA was processed
40 tRNA was processed
41 tRNA was processed
42 tRNA was processed
43 tRNA was processed
44 tRNA was processed
45 tRNA was processe

### Import refolded 
Check the correct results you want to import

In [18]:
with open (f"{output_dir}/non_canonical_for_refolding_{constrain}_results.txt", "r") as refolded:
    
    records = []
    line_counter = 0

    
    for line in refolded.readlines():

        
        if line.startswith(">"):
            
            superphyla, id, name = line.split("__")
            line_counter += 1
            
        elif line_counter == 1:
            
            seq = line
            line_counter += 1
        
        elif line_counter == 2:
            
            refolded_sec_struct = line.split(" ")[0]            

            records.append({
            'superphyla': superphyla[1:],
            "id": id,
            'name': name.strip(),
            "seq": seq.strip(),
            "refolded_sec_struct": refolded_sec_struct.strip()

            })
            line_counter = 0

df_refolded = pd.DataFrame(records)
df_refolded = df_refolded.drop_duplicates(subset=["superphyla", "name", "id"])
df_refolded.to_csv(f"./non_canon_refolding_{constrain}/non_canonical_for_refolding_{constrain}_refolded.csv", index=False)

### Abstract structures

In [19]:
struct_abs_list = []
stem_count_list = []
check_struct_list = []
for struct in df_refolded["refolded_sec_struct"]:
    struct_abs = struct_abstractor(struct)
    struct_abs_list.append(struct_abs)
    check_struct_list.append(check_struc_abs(struct_abs))
    stem_count_list.append(count_stems(struct_abs))
    
df_refolded["tRNA_structure_abstracted_refolded"] = struct_abs_list
df_refolded["stem_count_refolded"] = stem_count_list
df_refolded["canonical_refolded"] = check_struct_list

In [20]:
df_refolded

,superphyla,id,name,seq,refolded_sec_struct,tRNA_structure_abstracted_refolded,stem_count_refolded,canonical_refolded
0,Asgard,3045163_7,JASEJC010000005.trna5,GGGGCAUUGGUGUAAUGGUAGCACCCCUGGUCUGGCACCuAGGAGG...,(((((...(((((.......)))))(((((........)))))..(...,(.(.)(.).(.)).,4,tRNA_out
1,Euryarchaeota,2026739_441,JAENZL010000013.trna4,GCCAAGAUGGCGGAGCGGCUACGCAAUCGCCUGCAGAGCGAUACCa...,((((((..((((..(((....)))...))))(((...))).........,(.(.(.).)(.).(.).).,5,tRNA_out
2,Euryarchaeota,2026739_441,JAENZL010000028.trna1,AGCCCGGUAGUGUAGUGGUcaAUCAUGCGGGACUCUGACUCCCGCA...,(((((((((..(...(((.......(((((((.......)))))))...,(.(.(.(.).).).).,4,tRNA_out
3,Asgard,2053489_161,JAGXOA010000009.trna3,GCAGGUGUAGCCUAGCUGGUAAGACGCUGGCCUUGAGAGCCUGUGC...,(((((((..(((.(((.(......))))))).(((((..((((.((...,(.(.(.(.))).(.(.(.).).).).,7,tRNA_out
4,Asgard,2053489_161,JAGXOA010000027.trna3,GCGGGAAUGGCUGAGUGGUaAAAGCAAUAGAUUCAAAAUCUAUCCU...,((((((...(((..........))).(((((((...)))))))(((...,(.(.).(.)(.).(.).).,5,tRNA_out
...,...,...,...,...,...,...,...,...
129,Asgard,2053491_86,JAGXKG010000001.trna14,GCGGCCGUGGUCUAGCAUGGUUAGGACUGAAGCUUCCCAAGCUUCC...,((((((((((((.(((.(((..((........))..))).)))......,(.(.(.(.).).)..).,4,tRNA_out
130,Asgard,2053491_86,JAGXKG010000001.trna18,GCCCCAGUAGCUCAGCCuGGUuAGAUCGCAUCCUUGGUAAGGAUGA...,(((((((.......((((..........(((((((...))))))))...,(.(.(.)).(.)).,4,tRNA_out
131,Asgard,2053491_86,JAGXKG010000001.trna21,GCCGAGGUAGCCAAGCCcGGCcaACGGCGACGGACUCAAGAUCCGU...,(((((((..(((......)))....((.((.(((((((.....(((...,(.(.).(.(.(.(.(.).)).)).).,7,tRNA_out
132,Asgard,2053491_86,JAGXKG010000001.trna26,GAAUGUAGUCcGGCccAGCAUGGGGGACUCUCGAUCCUCCGACCCG...,....((..(((((((........(((((....(((((........)...,.(.(.(.(.).)).).,4,tRNA_out


### Apply filters if needed

In [ ]:
# size filtering & abstract structures
df_refolded_filtered = df_refolded[df_refolded["canonical_refolded"].isin(["tRNA_4-stems", "tRNA_5-stems"])]
# df_refolded_filtered = df_refolded_filtered[df_refolded_filtered["seq"].str.len() < 106]

# filter out pseudogenes
# trnascan_df = extract_tRNAscan_file("./")

# # filter out pseudogenes
# trnascan_df_filtered = trnascan_df[~trnascan_df["note"].str.contains("pseudo", case=False, na=False)]

# # combine data frames
# # drop double columns before merging
# trnascan_df_filtered_dropped = trnascan_df_filtered.drop(columns=["anticodon", "trna_type"], axis=1)

# df_merged = pd.merge(
#     df_refolded_filtered,
#     trnascan_df_filtered_dropped,
#     on=["superphyla", "id", "name"],
#     how="inner",
#     suffixes=('', '_trnascan') # if needed
# )

# # # combine data frames
# df_merged_refolded = pd.merge(
#     df_refolded_filtered,
#     trnascan_df,
#     on=["superphyla", "id", "name"],
#     how="inner"
# )
# df_merged_refolded

In [22]:
df_refolded_filtered

,superphyla,id,name,seq,refolded_sec_struct,tRNA_structure_abstracted_refolded,stem_count_refolded,canonical_refolded
44,DPANN,2250274_771,DAHXDH010000004.trna1,GGGGAGUUACGCUAACCuGGCAGACUUCCAGCCUUGGGCGCUGGAU...,(((((((...((((...))))......(((((.......))))).....,(.(.).(.).(.)).,4,tRNA_4-stems
